In [17]:
import os
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from utilities import *

Loading in Files

In [18]:
path = "./data"

In [19]:
filesNames = [x for x in os.listdir(path) if ".mat" in x]

In [20]:
filesNames

['S1.mat',
 'S3.mat',
 'S2.mat',
 'S6.mat',
 'S7.mat',
 'S5.mat',
 'S4.mat',
 'S8.mat']

In [21]:
labelDictionay = {10:"start Block Event",
                 1 : "Rest Cue",
                 3: "Right Arm",
                 4: "Left Arm",
                 7: "Right Hand",
                 8: "Left Hand"}

In [54]:
X_train,Y = getAllData(filesNames)

In [55]:
X_train = X_train.reshape([X_train.shape[0],1,500,21])

In [56]:
print(X_train.shape)
print(Y.shape)

(5867, 1, 500, 21)
(5867, 6)


In [57]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [58]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 21), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        #self.fc1 = nn.Linear(248 * batch_size, 6)
        self.fc1 = nn.Linear(248, 6)
        self.softmax = nn.Softmax(dim = 2)
        

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        
        # FC Layer
        #x = x.view(-1, 248 * batch_size)
        x = x.view(-1, self.numFeatures(x) ,248)
        x = F.sigmoid(self.fc1(x))
        
        #return self.softmax(x)
        return x
    
    def numFeatures(self, x):
        return int(np.array(x.shape).prod() / 248)


#net = EEGNet().cuda(0)
net = EEGNet()
#print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)).cuda(0))))
criterion = nn.BCELoss(size_average = True)
optimizer = optim.Adam(net.parameters(), lr= 0.001, weight_decay=0)

#Source: https://github.com/aliasvishnu/EEGNet/blob/master/EEGNet-PyTorch.ipynb

/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


In [59]:
net

EEGNet(
  (conv1): Conv2d(1, 16, kernel_size=(1, 21), stride=(1, 1))
  (batchnorm1): BatchNorm2d(16, eps=False, momentum=0.1, affine=True, track_running_stats=True)
  (padding1): ZeroPad2d(padding=(16, 17, 0, 1), value=0)
  (conv2): Conv2d(1, 4, kernel_size=(2, 32), stride=(1, 1))
  (batchnorm2): BatchNorm2d(4, eps=False, momentum=0.1, affine=True, track_running_stats=True)
  (pooling2): MaxPool2d(kernel_size=2, stride=4, padding=0, dilation=1, ceil_mode=False)
  (padding2): ZeroPad2d(padding=(2, 1, 4, 3), value=0)
  (conv3): Conv2d(4, 4, kernel_size=(8, 4), stride=(1, 1))
  (batchnorm3): BatchNorm2d(4, eps=False, momentum=0.1, affine=True, track_running_stats=True)
  (pooling3): MaxPool2d(kernel_size=(2, 4), stride=(2, 4), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=248, out_features=6, bias=True)
  (softmax): Softmax()
)

In [60]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 64
    
    predicted = []
    
    for i in range(int(len(X)/batch_size)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        #inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        inputs = Variable(torch.from_numpy(X[s:e]))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    #inputs = Variable(torch.from_numpy(X).cuda(0))
    inputs = Variable(torch.from_numpy(X))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    predicted = predicted[0]
    Y = Y.argmax(axis = 1)
    predicted = predicted.argmax(axis = 1)
    
    for param in params:
        if param == 'acc':
            results.append(sum(Y == predicted) / Y.shape[0])
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

In [61]:
#Defining Partitions
X_train = X_train[0:200]
y_train = Y[0:200]

In [62]:
y_train.shape

(200, 6)

In [63]:
optimizer = optim.Adam(net.parameters(), lr= 1e-4, weight_decay=1e-5)

In [71]:
batch_size = 64
trainingLoss = []
for epoch in range(100):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(np.array([y_train[s:e]])*1.0)
        
        # wrap them in Variable
        #inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data[0]
    
    # Validation accuracy
    params = ["acc"]
    trainingLoss.append(running_loss)
    print (params)
    print ("Training Loss ", running_loss)
    print ("Train - ", evaluate(net, X_train, y_train, params))
    #print ("Validation - ", evaluate(net, X_val, y_val, params))
    #print ("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


['acc']
Training Loss  tensor(0.4201)
Train -  [0.575]

Epoch  1
['acc']
Training Loss  tensor(0.4194)
Train -  [0.58]

Epoch  2
['acc']
Training Loss  tensor(0.4186)
Train -  [0.58]

Epoch  3
['acc']
Training Loss  tensor(0.4179)
Train -  [0.58]

Epoch  4
['acc']
Training Loss  tensor(0.4171)
Train -  [0.58]

Epoch  5
['acc']
Training Loss  tensor(0.4163)
Train -  [0.58]

Epoch  6
['acc']
Training Loss  tensor(0.4156)
Train -  [0.58]

Epoch  7
['acc']
Training Loss  tensor(0.4148)
Train -  [0.58]

Epoch  8
['acc']
Training Loss  tensor(0.4141)
Train -  [0.58]

Epoch  9
['acc']
Training Loss  tensor(0.4133)
Train -  [0.58]

Epoch  10
['acc']
Training Loss  tensor(0.4126)
Train -  [0.58]

Epoch  11
['acc']
Training Loss  tensor(0.4118)
Train -  [0.58]

Epoch  12
['acc']
Training Loss  tensor(0.4110)
Train -  [0.58]

Epoch  13
['acc']
Training Loss  tensor(0.4103)
Train -  [0.58]

Epoch  14
['acc']
Training Loss  tensor(0.4095)
Train -  [0.58]

Epoch  15
['acc']
Training Loss  tensor(0.4

In [49]:
tr

NameError: name 'tr' is not defined